In [1]:
import keras
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
path = keras.utils.get_file('nietzsche.txt',
                            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('Corpus length:', len(text))

Using TensorFlow backend.
C:\Users\artm-\anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\artm-\anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\artm-\anaconda3\envs\PythonGPU\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\artm-\anaconda3\envs\Pytho

Corpus length: 600901


In [2]:
maxlen = 60 # Извлечение последовательностей по 60 символов
step = 3 # Новые последовательности выбираются через каждые 3 символа
sentences = [] # Хранение извлеченных последовательностей
next_chars = [] # Хранение целей (символов, следующих за последовательностями

In [3]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text))) # Список уникальных символов в корпусе
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) # Словарь, отображающий уникальные символы в их индексы в списке «chars»
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1 # Прямое кодирование символов в бинарные массивы

Number of sequences: 200281
Unique characters: 59
Vectorization...


In [4]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys
for epoch in range(1, 60): # Обучение модели в течение 60 эпох
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1) # Выполнение одной итерации обучения
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(400):# Генерация 400 символов, начиная с начального текста
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
        
        preds = model.predict(sampled, verbose=0)[0] # Выбор следующего символа
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        
        generated_text += next_char
        generated_text = generated_text[1:]
        
        sys.stdout.write(next_char)

epoch 1

Epoch 1/1
200281/200281 [==============================] - 74s 368us/step - loss: 1.9588
epoch 2
Epoch 1/1
200281/200281 [==============================] - 73s 367us/step - loss: 1.6163
epoch 3
Epoch 1/1
200281/200281 [==============================] - 74s 371us/step - loss: 1.5259
epoch 4
Epoch 1/1
200281/200281 [==============================] - 77s 385us/step - loss: 1.4797
epoch 5
Epoch 1/1
200281/200281 [==============================] - 77s 386us/step - loss: 1.4494
epoch 6
Epoch 1/1
200281/200281 [==============================] - 77s 387us/step - loss: 1.4261
epoch 7
Epoch 1/1
200281/200281 [==============================] - 79s 393us/step - loss: 1.4077
epoch 8
Epoch 1/1
200281/200281 [==============================] - 76s 381us/step - loss: 1.3931
epoch 9
Epoch 1/1
200281/200281 [==============================] - 73s 366us/step - loss: 1.38290s - loss: 1.383
epoch 10
Epoch 1/1
200281/200281 [==============================] - 74s 371us/step - loss: 1.3717
epoch 11
Epo